In [31]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("SetFit/subj")
dataset = dataset.remove_columns("label_text")
dataset = dataset['train'].shuffle(seed=42).select([i for i in list(range(6000))])
train_ds = Dataset.from_dict(dataset[0:5000])
validation_ds = Dataset.from_dict(dataset[5000:5500])
test_ds = Dataset.from_dict(dataset[5500:6000])



display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Using custom data configuration SetFit___subj-848ec40cb3887b72
Reusing dataset json (C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)
100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.46it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-80b8e4129e527f71.arrow


Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

,text,label
0,"on this morning , johnny wakes to his mother '...",0
1,"despite a quieter middle section , involving a...",1
2,"[ kidd ] can write dialogue , work skillfully ...",1
3,"abandoned and filled with a mysterious past , ...",0
4,the romantic triangle leads to a surprising co...,0
...,...,...
4995,"jacobi , the most fluent of actors , is given ...",1
4996,even if you do n't understand what on earth is...,1
4997,"gerry 's daughter , mattie , accidentally prin...",0
4998,a well paced and satisfying little drama that ...,1


In [32]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [33]:
#creating antonyms list using wordnet
def antonyms_list_creation(target_word):
    antonyms_list = []
    try:
        for ann in wordnet.synsets(target_word):
            for lm in ann.lemmas():
                if lm.antonyms():
                    antonyms_list.append(lm.antonyms()[0].name())
        #print (set(antonyms_list))
        antonyms_list = list(set(antonyms_list))
        #transform _ to space
        antonyms_list = [s.replace('_', ' ') for s in antonyms_list]


        antonyms_list
        antonyms_list = [s for s in antonyms_list if ' ' not in s]
        
    except Exception as e:
        pass
    
    return antonyms_list

In [34]:
#senticnet part
senticnet_df = pd.read_excel(r'C:\Users\DELL\Text_Augmentation\senticnet\senticnet.xlsx',header=0)

In [35]:
#transform _ to space

senticnet_df['CONCEPT'] = senticnet_df['CONCEPT'].str.replace('_', ' ')


senticnet_df.set_index('CONCEPT',inplace=True)

print(senticnet_df)

               INTROSPECTION  TEMPER  ATTITUDE  SENSITIVITY PRIMARY EMOTION  \
CONCEPT                                                                       
abandon               -0.165     0.0     0.000        0.000     #melancholy   
abase                  0.000     0.0    -0.890        0.000       #loathing   
abash                  0.000     0.0    -0.341       -0.442           #fear   
abashment              0.000     0.0    -0.329        0.000        #dislike   
abate                  0.000     0.0    -0.339       -0.551           #fear   
...                      ...     ...       ...          ...             ...   
zotob worm             0.000     0.0     0.000       -0.958         #terror   
zotob wrestle          0.000     0.0     0.000       -0.962         #terror   
zotob wriggle          0.000     0.0     0.000       -0.918         #terror   
zotob writhe           0.000     0.0     0.000       -0.979         #terror   
zz                     0.000     0.0    -0.182      

In [36]:
#creating opposite words list using senticnet
def opposite_polarity_words_list_creation(target_word):
    if target_word in senticnet_df.index:
        
        INTROSPECTION = -senticnet_df.loc[target_word, 'INTROSPECTION']
        TEMPER = -senticnet_df.loc[target_word, 'TEMPER']
        ATTITUDE = -senticnet_df.loc[target_word, 'ATTITUDE']
        SENSITIVITY = -senticnet_df.loc[target_word, 'SENSITIVITY']





        extracted_df = senticnet_df[(senticnet_df['INTROSPECTION'] == INTROSPECTION) & 
                                   (senticnet_df['TEMPER'] == TEMPER) & (senticnet_df['ATTITUDE'] == ATTITUDE) & 
                                    (senticnet_df['SENSITIVITY'] == SENSITIVITY)]


        opposite_polarity_words_list = extracted_df.index.values.tolist()
        opposite_polarity_words_list = [s for s in opposite_polarity_words_list if ' ' not in s]
    
    else:
        opposite_polarity_words_list = []
    
    return opposite_polarity_words_list

In [37]:
import itertools
from textattack.transformations import WordSwap
class Antipode_WordSwap(WordSwap):
    
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        antonyms_list = antonyms_list_creation(word)
        antipode_list = []
        try:
            antipode_list.append(opposite_polarity_words_list_creation(word))
        except Exception as e:
            pass
        
        antipode_list.append(antonyms_list)
        antipode_list = list(itertools.chain.from_iterable(antipode_list))
        antipode_list = list(set(antipode_list))

        return antipode_list


In [38]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore
from textattack.augmentation import Augmenter


transformation = Antipode_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(),
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=False, compare_against_original=True, language_nltk='eng', language_stanza='en')
                ]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!


In [39]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):

    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)

time_end = time.perf_counter()
tim = time_end - time_sta

In [40]:
print(tim/3600)




0.596944574361111


In [41]:
display(results_list)

[["on this morning , johnny wakes to his mother 's accusations that he has botched a simple scam , disinherit them without enough money to payoff their outdoors man ."],
 ["despite a quieter middle section , involving aragorn 's dreams of arwen , this is even better than the fellowship . there are scenes of cinematic perfection that azoth your heart home ."],
 ['[ kidd ] can write dialogue , frowzy skillfully with actors , and he has a pretty good handle on urban loneliness of the knowing , virulent new york city dismemberment .'],
 ['abandoned and filled with a mysterious past , holy angel school for boy is about to reveal its secrets of betrayal , jealousy , and relief .'],
 ["the romantic triangle leads to a surprising conclusion as the young man 's secret motive has nonexplosive consequences ."],
 ['as a film director , labute discontinue to improve .'],
 ['now , along with other surviors , jill valentine , claire redfield and carlos oliviera , they must wisteria to survive , to tr

In [42]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[["on this morning , johnny wakes to his mother 's accusations that he has botched a simple scam , leaving them without enough money to payoff their inside man ."],
 ["despite a quieter middle section , involving aragorn 's dreams of arwen , this is even better than the fellowship . there are scenes of cinematic perfection that steal your heart away ."],
 ['[ kidd ] can write dialogue , work skillfully with actors , and he has a pretty good handle on urban loneliness of the knowing , virulent new york city variety .'],
 ['abandoned and filled with a mysterious past , holy angel school for girls is about to reveal its secrets of betrayal , jealousy , and vengeance .'],
 ["the romantic triangle leads to a surprising conclusion as the young man 's secret motive has explosive consequences ."],
 ['as a film director , labute continues to improve .'],
 ['now , along with other surviors , jill valentine , claire redfield and carlos oliviera , they must fight to survive , to escape the nightma

In [43]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,"on this morning , johnny wakes to his mother '...","on this morning , johnny wakes to his mother '..."
1,"despite a quieter middle section , involving a...","despite a quieter middle section , involving a..."
2,"[ kidd ] can write dialogue , work skillfully ...","[ kidd ] can write dialogue , frowzy skillfull..."
3,"abandoned and filled with a mysterious past , ...","abandoned and filled with a mysterious past , ..."
4,the romantic triangle leads to a surprising co...,the romantic triangle leads to a surprising co...
...,...,...
4995,"jacobi , the most fluent of actors , is given ...","jacobi , the most fluent of actors , is starve..."
4996,even if you do n't understand what on earth is...,even if you do n't understand what on earth is...
4997,"gerry 's daughter , mattie , accidentally prin...","gerry 's daughter , mattie , accidentally prin..."
4998,a well paced and satisfying little drama that ...,a well paced and satisfying large drama that d...


In [44]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,"on this morning , johnny wakes to his mother '...","on this morning , johnny wakes to his mother '..."
1,"despite a quieter middle section , involving a...","despite a quieter middle section , involving a..."
2,"[ kidd ] can write dialogue , work skillfully ...","[ kidd ] can write dialogue , frowzy skillfull..."
3,"abandoned and filled with a mysterious past , ...","abandoned and filled with a mysterious past , ..."
4,the romantic triangle leads to a surprising co...,the romantic triangle leads to a surprising co...
...,...,...
4856,"jacobi , the most fluent of actors , is given ...","jacobi , the most fluent of actors , is starve..."
4857,even if you do n't understand what on earth is...,even if you do n't understand what on earth is...
4858,"gerry 's daughter , mattie , accidentally prin...","gerry 's daughter , mattie , accidentally prin..."
4859,a well paced and satisfying little drama that ...,a well paced and satisfying large drama that d...


In [45]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\augmented_datasets\Antipode with Pos Verb_Noun=False\run3\dataset.csv')